In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0" 
import ast
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras import backend as K
from keras import Model
from keras import optimizers
from keras.legacy import interfaces
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten, Activation, GlobalAveragePooling2D, BatchNormalization
from keras.metrics import categorical_accuracy, top_k_categorical_accuracy, categorical_crossentropy
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.optimizers import Adam

from keras.optimizers import SGD
from keras import callbacks
from data import *
from keras_helper import *

from keras.applications import DenseNet201
# from keras.applications.xception import preprocess_input

Using TensorFlow backend.


In [2]:
def preprocess_input(x):
    x /= 127.5
    x -= 1.
    return x

In [3]:
TOT = 35534421
EPOCHS = 150
size = 65
batchsize = 256
lw = 6
channel = 1
STEPS = TOT / EPOCHS / batchsize
NCATS = 340
mixup = 0
center = True
optimizer = 'adam'
cb = 'raw'

In [4]:
K.clear_session()
model = DenseNet201(input_shape=(size, size, 1), weights=None, classes=NCATS)

if optimizer == 'sgd':
    opt = SGD(0.002, momentum=0.9, nesterov=True)
    opt = NormalizedOptimizer(opt, normalization='l2')
elif optimizer == 'adam':
    opt = Adam(lr=0.002)

model.compile(optimizer=opt, loss='categorical_crossentropy',
              metrics=[categorical_crossentropy, categorical_accuracy, top_3_accuracy])
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 65, 65, 1)    0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 71, 71, 1)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 33, 33, 64)   3136        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 33, 33, 64)   256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

In [5]:
valid_df = pd.read_csv('../input/valid.csv')
x_valid = df_to_image_array_xd(valid_df, size, lw=lw, preprocess_input=preprocess_input, 
                               channel=channel, center = center)
y_valid = keras.utils.to_categorical(valid_df.word, num_classes=NCATS)

In [6]:
print(x_valid.shape, y_valid.shape)
print('Validation array memory {:.2f} GB'.format(x_valid.nbytes / 1024.**3 ))

(34000, 65, 65, 1) (34000, 340)
Validation array memory 0.54 GB


In [7]:
train_datagen = image_generator_xd(df_path = '../input/train_all_balance.csv', 
                                   size=size, batchsize=batchsize, lw=lw, 
                                   preprocess_input=preprocess_input,
                                  channel=channel, mixup=mixup, center=center)

In [8]:
model_prefix = 'dense201_{}_lw{}_balance_mix{}_{}_{}'.format(size, lw, mixup, optimizer, cb)
if center:
    model_prefix += '_center'
print(model_prefix)
check_path = "./models/{}.model".format(model_prefix)
fold = 9
if cb == 'raw':
    callbks = [
        ReduceLROnPlateau(monitor='val_top_3_accuracy', factor=0.5, patience=5,
                          min_delta=0.005, mode='max', cooldown=3, verbose=1),
        callbacks.ModelCheckpoint(check_path,
                                    monitor='val_top_3_accuracy', 
                                    mode = 'max', save_best_only=True, verbose=1),
        callbacks.TensorBoard(log_dir='./log/{}'.format(model_prefix)),
        EarlyStopping(monitor='val_top_3_accuracy', mode='max',patience=20, verbose=1)
    ]
elif cb == 'snap':
    snapshot = SnapshotCallbackBuilder(nb_epochs=EPOCHS,
                                       nb_snapshots=1,
                                       size = size,
                                       init_lr=1e-3, 
                                       fold=fold)

    callbks = snapshot.get_callbacks(model_prefix = model_prefix)

dense201_65_lw6_balance_mix0_adam_raw_center


In [9]:
# model.load_weights('models/xception_raw_deep71.model')

In [ ]:
hists = []
hist = model.fit_generator(
    train_datagen, steps_per_epoch=STEPS, epochs=EPOCHS, verbose=1,
    validation_data=(x_valid, y_valid),
    callbacks = callbks 
)

Epoch 1/150
926/925 [==============================] - 353s 382ms/step - loss: 2.3176 - categorical_crossentropy: 2.3176 - categorical_accuracy: 0.4676 - top_3_accuracy: 0.6658 - val_loss: 5.8687 - val_categorical_crossentropy: 5.8687 - val_categorical_accuracy: 0.1441 - val_top_3_accuracy: 0.2775

Epoch 00001: val_top_3_accuracy improved from -inf to 0.27747, saving model to ./models/dense201_65_lw6_balance_mix0_adam_raw_center.model
Epoch 2/150
926/925 [==============================] - 318s 344ms/step - loss: 1.5861 - categorical_crossentropy: 1.5861 - categorical_accuracy: 0.6111 - top_3_accuracy: 0.8026 - val_loss: 3.3021 - val_categorical_crossentropy: 3.3021 - val_categorical_accuracy: 0.3333 - val_top_3_accuracy: 0.5324

Epoch 00002: val_top_3_accuracy improved from 0.27747 to 0.53244, saving model to ./models/dense201_65_lw6_balance_mix0_adam_raw_center.model
Epoch 3/150
926/925 [==============================] - 319s 344ms/step - loss: 1.4270 - categorical_crossentropy: 1.427

926/925 [==============================] - 319s 344ms/step - loss: 0.9749 - categorical_crossentropy: 0.9749 - categorical_accuracy: 0.7506 - top_3_accuracy: 0.8977 - val_loss: 1.0202 - val_categorical_crossentropy: 1.0202 - val_categorical_accuracy: 0.7393 - val_top_3_accuracy: 0.8905

Epoch 00021: val_top_3_accuracy did not improve from 0.89344
Epoch 22/150
926/925 [==============================] - 318s 344ms/step - loss: 0.9653 - categorical_crossentropy: 0.9653 - categorical_accuracy: 0.7526 - top_3_accuracy: 0.8995 - val_loss: 1.0132 - val_categorical_crossentropy: 1.0132 - val_categorical_accuracy: 0.7398 - val_top_3_accuracy: 0.8927

Epoch 00022: ReduceLROnPlateau reducing learning rate to 0.0010000000474974513.

Epoch 00022: val_top_3_accuracy did not improve from 0.89344
Epoch 23/150
926/925 [==============================] - 319s 344ms/step - loss: 0.9082 - categorical_crossentropy: 0.9082 - categorical_accuracy: 0.7686 - top_3_accuracy: 0.9074 - val_loss: 0.9067 - val_categ

In [ ]:
test = pd.read_csv('../input/test_simplified.csv')
test.head()
x_test = df_to_image_array_xd(test, size, lw=lw, 
                              preprocess_input=preprocess_input,
                             channel=channel, center = center)
print(test.shape, x_test.shape)
print('Test array memory {:.2f} GB'.format(x_test.nbytes / 1024.**3 ))

np_classes = np.load('../input/classes.npy')
id2cat = {k: cat.replace(' ', '_') for k, cat in enumerate(np_classes)}


In [ ]:
def doodle_predict(model, model_path, x_test):
    model.load_weights(model_path)

    test_predictions = model.predict(x_test, batch_size=128, verbose=1)
    top3 = preds2catids(test_predictions)
    top3cats = top3.replace(id2cat)
    test['word'] = top3cats['a'] + ' ' + top3cats['b'] + ' ' + top3cats['c']
    submission = test[['key_id', 'word']]

    import kaggle_util
    kaggle_util.save_result(submission,  
                            '../result/{}.csv'.format(model_prefix), 
                            'quickdraw-doodle-recognition', 
                            send=True, index=False)

In [ ]:
doodle_predict(model, check_path, x_test)